In [2]:
import os, datetime
from tqdm import tqdm

import numpy as np
import pandas as pd

In [ ]:
data_dir = '../data/'

df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
# df_test = pd.read_csv(os.path.join(data_dir, 'test.csv'))
# df_revealed_test = pd.read_csv(os.path.join(data_dir, 'revealed_test.csv'))
df_submission = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))
df_census = pd.read_csv(os.path.join(data_dir, 'census_starter.csv'))

In [7]:
df_all_revealed_test = pd.read_csv(os.path.join(data_dir, 'all_revealed_data.csv'))

In [8]:
df_all_revealed_test

,row_id,microbusiness_density
0,1001_2022-11-01,3.292227
1,1003_2022-11-01,7.941944
2,1005_2022-11-01,1.235309
3,1007_2022-11-01,1.275281
4,1009_2022-11-01,1.803057
...,...,...
25075,56037_2023-06-01,2.995494
25076,56039_2023-06-01,25.593407
25077,56041_2023-06-01,4.152722
25078,56043_2023-06-01,3.200000


In [17]:
df_lastvalues = df_train.drop_duplicates(subset='cfips', keep='last')
df_lastvalues.head()

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
38,1001_2022-10-01,1001,Autauga County,Alabama,2022-10-01,3.463856,1472
77,1003_2022-10-01,1003,Baldwin County,Alabama,2022-10-01,8.359798,14320
116,1005_2022-10-01,1005,Barbour County,Alabama,2022-10-01,1.232074,244
155,1007_2022-10-01,1007,Bibb County,Alabama,2022-10-01,1.287240,229
194,1009_2022-10-01,1009,Blount County,Alabama,2022-10-01,1.831783,813


3.4638555

In [30]:
for i, row in tqdm(df_submission.iterrows(), total=df_submission.shape[0]):
	cfips = row.row_id.split('_')[0]
	lastvalue = df_lastvalues[df_lastvalues['cfips'] == int(cfips)].microbusiness_density.item()
	df_submission.iloc[i, 1] = lastvalue

100%|██████████| 25080/25080 [00:06<00:00, 3910.41it/s]


In [31]:
df_submission

,row_id,microbusiness_density
0,1001_2022-11-01,3.463856
1,1003_2022-11-01,8.359798
2,1005_2022-11-01,1.232074
3,1007_2022-11-01,1.287240
4,1009_2022-11-01,1.831783
...,...,...
25075,56037_2023-06-01,2.823801
25076,56039_2023-06-01,26.273220
25077,56041_2023-06-01,4.009369
25078,56043_2023-06-01,3.126551


In [44]:
# define evaluation metric, SMAPE
def SMAPE(actuals, preds):
	smape = 0
	for actual, pred in zip(actuals, preds):
		if (actual==0) and (pred==0): continue
		smape += abs(actual-pred) / (abs(actual) + abs(pred)) * 2 * 100
	return smape / len(actuals)

smape_value = SMAPE(df_all_revealed_test.microbusiness_density.values, df_submission.microbusiness_density.values)
smape_value

np.float64(5.083047455230122)

In [50]:
df_revealed_test.first_day_of_month.unique()

array(['2022-11-01', '2022-12-01'], dtype=object)

In [55]:
submission = df_submission[df_submission.row_id.str.contains('2022')]

In [54]:
df_revealed_test

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2022-11-01,1001,Autauga County,Alabama,2022-11-01,3.442677,1463
1,1001_2022-12-01,1001,Autauga County,Alabama,2022-12-01,3.470915,1475
2,1003_2022-11-01,1003,Baldwin County,Alabama,2022-11-01,8.257636,14145
3,1003_2022-12-01,1003,Baldwin County,Alabama,2022-12-01,8.250630,14133
4,1005_2022-11-01,1005,Barbour County,Alabama,2022-11-01,1.247223,247
...,...,...,...,...,...,...,...
6265,56041_2022-12-01,56041,Uinta County,Wyoming,2022-12-01,4.050703,588
6266,56043_2022-11-01,56043,Washakie County,Wyoming,2022-11-01,3.143093,190
6267,56043_2022-12-01,56043,Washakie County,Wyoming,2022-12-01,3.060380,185
6268,56045_2022-11-01,56045,Weston County,Wyoming,2022-11-01,1.785395,100


In [ ]:
# sort new test data along the order of submission
sortmap = {val:idx for idx, val in enumerate(df_revealed_test.row_id.values)}
submission['sortmap'] = submission.row_id.map(sortmap)
 = submission.sort_values('sortmap').drop(columns='sortmap')

/var/folders/jz/wsn3kwpx7cq0nsrxkk2g7nrr0000gn/T/ipykernel_93436/507135792.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['sortmap'] = submission.row_id.map(sortmap)


In [58]:
smape_value = SMAPE(df_revealed_test.microbusiness_density.values, submission.microbusiness_density.values)
smape_value

np.float64(1.7862393745117036)

The most basic last value forecasting outperforms in short period of time like only 2022 but not good in a long term like all test period (2022nov-2023june)

# Adjusting microbusiness density with 2021 census data

Based on this notebook, 2023 density will refer 2021 census to calculate its value.

In [61]:
COLS = ['GEO_ID','NAME','S0101_C01_026E']
df2020 = pd.read_csv(os.path.join(data_dir, 'census', 'ACSST5Y2020.S0101-Data.csv'),usecols=COLS)
df2020 = df2020.iloc[1:]
df2020['S0101_C01_026E'] = df2020['S0101_C01_026E'].astype('int')
print(df2020.shape)
df2020.head()

(3221, 3)


/var/folders/jz/wsn3kwpx7cq0nsrxkk2g7nrr0000gn/T/ipykernel_93436/3151360393.py:2: DtypeWarning: Columns (102) have mixed types. Specify dtype option on import or set low_memory=False.
  df2020 = pd.read_csv(os.path.join(data_dir, 'census', 'ACSST5Y2020.S0101-Data.csv'),usecols=COLS)


,GEO_ID,NAME,S0101_C01_026E
1,0500000US01001,"Autauga County, Alabama",42496
2,0500000US01003,"Baldwin County, Alabama",171296
3,0500000US01005,"Barbour County, Alabama",19804
4,0500000US01007,"Bibb County, Alabama",17790
5,0500000US01009,"Blount County, Alabama",44383


In [62]:
df2021 = pd.read_csv(os.path.join(data_dir, 'census', 'ACSST5Y2021.S0101-Data.csv'), usecols=COLS)
df2021 = df2021.iloc[1:]
df2021['S0101_C01_026E'] = df2021['S0101_C01_026E'].astype('int')
print( df2021.shape )
df2021.head()

(3221, 3)


/var/folders/jz/wsn3kwpx7cq0nsrxkk2g7nrr0000gn/T/ipykernel_93436/1548430182.py:1: DtypeWarning: Columns (102) have mixed types. Specify dtype option on import or set low_memory=False.
  df2021 = pd.read_csv(os.path.join(data_dir, 'census', 'ACSST5Y2021.S0101-Data.csv'), usecols=COLS)


,GEO_ID,NAME,S0101_C01_026E
1,0500000US01001,"Autauga County, Alabama",44438
2,0500000US01003,"Baldwin County, Alabama",178105
3,0500000US01005,"Barbour County, Alabama",19995
4,0500000US01007,"Bibb County, Alabama",17800
5,0500000US01009,"Blount County, Alabama",45201


In [63]:
df2020['cfips'] = df2020.GEO_ID.apply(lambda x: int(x.split('US')[-1]) )
adult2020 = df2020.set_index('cfips').S0101_C01_026E.to_dict()

df2021['cfips'] = df2021.GEO_ID.apply(lambda x: int(x.split('US')[-1]) )
adult2021 = df2021.set_index('cfips').S0101_C01_026E.to_dict()

In [71]:
df_submission['cfips'] = df_submission.row_id.apply(lambda x: int(x.split('_')[0]))

In [72]:
df_submission['adult2020'] = df_submission.cfips.map(adult2020)
df_submission['adult2021'] = df_submission.cfips.map(adult2021)
df_submission.head()

,row_id,microbusiness_density,cfips,adult2020,adult2021
0,1001_2022-11-01,3.463856,1001,42496,44438
1,1003_2022-11-01,8.359798,1003,171296,178105
2,1005_2022-11-01,1.232074,1005,19804,19995
3,1007_2022-11-01,1.287240,1007,17790,17800
4,1009_2022-11-01,1.831783,1009,44383,45201


In [73]:
df_submission['adjusted_microbusiness_density'] = df_submission['microbusiness_density'] * df_submission['adult2020'] / df_submission['adult2021']

In [75]:
df_submission['adjusted_microbusiness_density']

0         3.312480
1         8.040201
2         1.220305
3         1.286517
4         1.798633
           ...    
25075     2.871739
25076    26.266367
25077     3.975138
25078     3.150000
25079     1.818512
Name: adjusted_microbusiness_density, Length: 25080, dtype: float64

In [76]:
smape_value = SMAPE(df_all_revealed_test.microbusiness_density.values, df_submission.adjusted_microbusiness_density.values)
smape_value

np.float64(3.8673410842826885)

# 위에 내용 정리해서 올리기